<a href="https://colab.research.google.com/github/willschuerman/lets-stop-wildfires-hackathon/blob/will/wildfire_smoke_challenge_1A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


ModuleNotFoundError: ignored

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate

In [0]:
import tarfile
import pandas
from os import listdir
import numpy as np
import glob
from datetime import datetime
from PIL import Image
import matplotlib.pyplot as plt

In [0]:
bs = 64

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# 1. Load data
This loads the 1.4GB dataset from challenge 2, which includes all the files. 


In [7]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rRKIGPPeLognGoQMJ6AiDiwj83_aMVmY' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1rRKIGPPeLognGoQMJ6AiDiwj83_aMVmY" -O train_data_set.tar.gz && rm -rf /tmp/cookies.txt

--2020-01-14 16:23:19--  https://docs.google.com/uc?export=download&confirm=ex9g&id=1rRKIGPPeLognGoQMJ6AiDiwj83_aMVmY
Resolving docs.google.com (docs.google.com)... 74.125.142.100, 74.125.142.139, 74.125.142.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pf6gla3t4fdr1frl5j8dj44rfjjehibu/1579017600000/17654960937288161688/*/1rRKIGPPeLognGoQMJ6AiDiwj83_aMVmY?e=download [following]
--2020-01-14 16:23:20--  https://doc-08-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pf6gla3t4fdr1frl5j8dj44rfjjehibu/1579017600000/17654960937288161688/*/1rRKIGPPeLognGoQMJ6AiDiwj83_aMVmY?e=download
Resolving doc-08-24-docs.googleusercontent.com (doc-08-24-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-08-24-docs.googleusercontent.com

In [0]:
tarred_file = tarfile.open("train_data_set.tar.gz")
tarred_file.extractall()
tarred_file.close()

In [0]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [0]:
camera_folders = listdir_nohidden('train_data_set')
# for now, leaving out image trains which do not include any smoke pictures
no_smoke_folders = ['train_data_set/20191018-wilson-w-mobo-c ','train_data_set/20191023-starr-n-mobo-c ','train_data_set/20191101-bh-e-mobo-c','train_data_set/20191101-bm-e-mobo-c','train_data_set/20191102-bm-e-mobo-c-2 ']
camera_folders = [x for x in camera_folders if x not in no_smoke_folders]

In [11]:
camera_names = [x[24:] for x in camera_folders]
camera_names

['Caliente-om-w-mobo',
 'Topanga-69bravo-n-mobo',
 'Cranston-hp-n-mobo-c',
 'RockHouse-wc-e-mobo-c',
 'MLOSouth-lo-s-mobo-c',
 'Whittier-syp-n-mobo-c',
 'BBM-bm-n-mobo',
 'MLOSouth2-lo-s-mobo-c',
 'PinosSouth-pi-s-mobo-c',
 '94Fire-lp-s-mobo-c',
 'OtaySouth-om-s-mobo',
 'Hope-wc-e-mobo-c',
 'Bridle-hp-n-mobo-c',
 'MLOSouth3-lo-s-mobo-c',
 'Benton-hp-n-mobo-c',
 'fallbrook-rm-w-mobo-c',
 'Lilac-rm-s-mobo',
 'West-lp-n-mobo-c']

#2. Generate CSV with info about images

In [12]:
# load first folder of images
image_fpaths = listdir_nohidden(camera_folders[0])
d = {'fPath':image_fpaths}
image_info = pd.DataFrame(d)
image_info['fName'] = image_info['fPath'].str[-21:]
cond1 = image_info['fName'].str[11] == '+'
image_info['isSmoke'] = np.where(cond1, 1, 0)
image_info.head()

# mark image in which smoke first appears
first_smoke_images = ['1498416633_+00240.jpg',
'1499546383_+00120.jpg',
'1512674164_+00180.jpg',
'1528757466_+00060.jpg',
'1529020885_+00180.jpg',
'1529001020_+00060.jpg',
'1530739165_+00120.jpg',
'1532544985_+00060.jpg',
'1530901921_+00240.jpg',
'1559167445_+00060.jpg',
'1562959460_+00119.jpg',
'1563135083_+00060.jpg',
'1563122707_+00120.jpg',
'1563220703_-02100.jpg',
'1563228743_+01620.jpg',
'1564685261_+00060.jpg',
'1564869279_+00060.jpg',
'1565731242_+00000.jpg'] # easier than importing from a separate CSV file

# mark image in which smoke first appears
cond1 = image_info.fName.isin(first_smoke_images)
image_info['firstSmoke'] = np.where(cond1, 1, 0)

# repeat for all other folders and concatenate
for fld in range(1,len(camera_folders)):
  image_fpaths = listdir_nohidden(camera_folders[fld])
  d = {'fPath':image_fpaths}
  tmp = pd.DataFrame(d)
  tmp['fName'] = tmp['fPath'].str[-21:]
  cond1 = tmp['fName'].str[11] == '+'
  tmp['isSmoke'] = np.where(cond1, 1, 0)
  cond1 = tmp.fName.isin(first_smoke_images)
  tmp['firstSmoke'] = np.where(cond1, 1, 0)
  image_info = pd.concat([image_info,tmp])

# extract name of area
image_info['camera_name'] = image_info['fPath'].str.split('/',expand = True)[1]
image_info['camera_name'] = image_info.camera_name.str.split('-',expand=True)[1]

# add in time info
image_info['unixTime'] =image_info['fName'].str[0:10].astype(int)

image_info['dateTime'] = pd.to_datetime(image_info['unixTime'],unit='s')
image_info = image_info.sort_values(by=['camera_name','unixTime'])

# fix the index after sorting and concatenating
image_info.index = pd.RangeIndex(len(image_info.index)) 
image_info.tail()

,fPath,fName,isSmoke,firstSmoke,camera_name,unixTime,dateTime
1334,train_data_set/20180611-fallbrook-rm-w-mobo-c/...,1528759566_+02160.jpg,1,0,fallbrook,1528759566,2018-06-11 23:26:06
1335,train_data_set/20180611-fallbrook-rm-w-mobo-c/...,1528759626_+02220.jpg,1,0,fallbrook,1528759626,2018-06-11 23:27:06
1336,train_data_set/20180611-fallbrook-rm-w-mobo-c/...,1528759686_+02280.jpg,1,0,fallbrook,1528759686,2018-06-11 23:28:06
1337,train_data_set/20180611-fallbrook-rm-w-mobo-c/...,1528759746_+02340.jpg,1,0,fallbrook,1528759746,2018-06-11 23:29:06
1338,train_data_set/20180611-fallbrook-rm-w-mobo-c/...,1528759806_+02400.jpg,1,0,fallbrook,1528759806,2018-06-11 23:30:06


#3. Load images, subtract RGB, write to new folder
This currently takes a long time because of the for loops, if anyone has suggestions on speeding it up would love to hear about it. 

In [0]:
%%time
# takes about 40 seconds per directory, longest duration cell in notebook so far because of the i/o. 
# This could be sped up by writing subtraction to binary .npy files...

!rm -rd train_data_set_subtract
# create a new directory for the subtraction images
os.makedirs('train_data_set_subtract',exist_ok=True) 

# extract the name of the image to generate new filenames and subset data
camNames = image_info['camera_name'].unique()
cN = camNames[0]

# subset to one set of images
tmp_df = image_info.loc[image_info.camera_name==cN]
nimages = tmp_df.shape[0]

# create lists in which to append the new names and data info.
# for a2b in range(0,nimages-1):
for a2b in range(0,nimages-1):
  # read in pairs of images
  img1 = np.array(Image.open(tmp_df['fPath'].iloc[a2b]))
  img2 = np.array(Image.open(tmp_df['fPath'].iloc[a2b+1]))
  # subtract image 1 from image 2, convert to image
  img_subtraction = img2-img1
  img_subtraction = Image.fromarray(img_subtraction)
  # generate new unique name that contains all relevant file info
  fTime = np.round(np.mean([tmp_df['unixTime'].iloc[a2b+1],tmp_df['unixTime'].iloc[a2b]]))
  # get info about the two images and append to a list
  img1_smoke = tmp_df['isSmoke'].iloc[a2b]
  img2_smoke = tmp_df['isSmoke'].iloc[a2b+1]

  if img1_smoke ==1 and img2_smoke==1:
    smokeInfo = ('Smoke')
  elif img1_smoke==0 and img2_smoke==1:
    smokeInfo = ('smokeStart')
  elif img1_smoke==1 and img2_smoke==0:
    smokeInfo = ('smokeStop')
  else:
    smokeInfo = ('noSmoke')

  new_fname = cN + '_' + str(int(fTime)) + '_' + smokeInfo + '_rgbsub.jpg'
  img_subtraction.save('train_data_set_subtract/' + new_fname)


# repeat for all camera folders
for d in range(1,len(camNames)):
  cN = camNames[d]

  # subset to one set of images
  tmp_df = image_info.loc[image_info.camera_name==cN]
  nimages = tmp_df.shape[0]

  # create lists in which to append the new names and data info.
  # for a2b in range(0,nimages-1):
  for a2b in range(0,nimages-1):
    # read in pairs of images
    img1 = np.array(Image.open(tmp_df['fPath'].iloc[a2b]))
    img2 = np.array(Image.open(tmp_df['fPath'].iloc[a2b+1]))
    # subtract image 1 from image 2, convert to image
    img_subtraction = img2-img1
    img_subtraction = Image.fromarray(img_subtraction)
    # generate new unique name that contains all relevant file info
    fTime = np.round(np.mean([tmp_df['unixTime'].iloc[a2b+1],tmp_df['unixTime'].iloc[a2b]]))
    # get info about the two images and append to a list
    img1_smoke = tmp_df['isSmoke'].iloc[a2b]
    img2_smoke = tmp_df['isSmoke'].iloc[a2b+1]

    if img1_smoke ==1 and img2_smoke==1:
      smokeInfo = ('Smoke')
    elif img1_smoke==0 and img2_smoke==1:
      smokeInfo = ('smokeStart')
    elif img1_smoke==1 and img2_smoke==0:
      smokeInfo = ('smokeStop')
    else:
      smokeInfo = ('noSmoke')

    new_fname = cN + '_' + str(int(fTime)) + '_' + smokeInfo + '_rgbsub.jpg'
    img_subtraction.save('train_data_set_subtract/' + new_fname)

In [67]:
# generate a new dataframe with all the info extracted from the newly generated filenames
fNames = os.listdir('train_data_set_subtract')
sub_info = pd.DataFrame(fNames,columns = ['fName'])
tmp = sub_info['fName'].str.split('_',expand=True) 
sub_info['camera'] = tmp[0]
sub_info['unixTime'] = tmp[1].astype(int)
sub_info['dateTime'] = pd.to_datetime(sub_info['unixTime'],unit='s')
sub_info['smokeLabel'] = tmp[2]
sub_info = sub_info.sort_values(by=['camera','unixTime'])
sub_info.head()


,fName,camera,unixTime,dateTime,smokeLabel
822,94Fire_1559166035_noSmoke_rgbsub.jpg,94Fire,1559166035,2019-05-29 21:40:35,noSmoke
1039,94Fire_1559166095_noSmoke_rgbsub.jpg,94Fire,1559166095,2019-05-29 21:41:35,noSmoke
1028,94Fire_1559166155_noSmoke_rgbsub.jpg,94Fire,1559166155,2019-05-29 21:42:35,noSmoke
1027,94Fire_1559166215_noSmoke_rgbsub.jpg,94Fire,1559166215,2019-05-29 21:43:35,noSmoke
855,94Fire_1559166275_noSmoke_rgbsub.jpg,94Fire,1559166275,2019-05-29 21:44:35,noSmoke
